In [30]:
# define model
def run_model(dependcon,df):
    import time
    import hddm

    samplenum = 500
    burn = 200
    name1 = "".join([x for x in dependcon.keys()])
    postfix = ".hddm"
    p_outlier = 0.05

    starttime = time.time()
    m = hddm.HDDM(df,p_outlier=p_outlier,
    depends_on=dependcon)
    m.find_starting_values()
    m.sample(samplenum,burn,dbname=name1+'traces.db', db='pickle')
    m.save(name1+postfix)
    del m
    timegap = time.time()-starttime
    print(name1 + 'usage time: %.3f min' %(timegap/60))

In [31]:
# import data and preparation
import pandas as pd
import hddm
df = pd.read_csv(
  "/home/mw/data222.csv",
  header="infer",encoding="utf-8")
df = df.loc[df["resp"]!="NONE"]
df = hddm.utils.flip_errors(df.rename(columns={"ACC":"response","sub":"subj_idx","RT":"rt"}))
df.tail()


,subj_idx,response,rt,resp,Cresp,PresentOrder,condition,playsound,key_state,t_resp
4015,S28,0,-0.584423,j,f,128,1,1,0,0.60
4017,S28,0,-0.610923,f,j,130,4,1,0,0.62
4020,S28,0,-0.615764,f,j,133,4,1,0,0.66
4025,S28,1,0.611960,f,f,138,1,0,0,0.70
4027,S28,1,0.513033,j,j,140,4,0,0,0.68


In [32]:
# define parallel computation
def parallel(func, *args, show=False, thread=False, **kwargs):
    """
    并行计算
    :param func: 函数，必选参数
    :param args: list/tuple/iterable,1个或多个函数的动态参数，必选参数
    :param show:bool,默认False,是否显示计算进度
    :param thread:bool,默认False,是否为多线程
    :param kwargs:1个或多个函数的静态参数，key-word形式
    :return:list,与函数动态参数等长
    """
    import time
    from functools import partial
    from pathos.pools import ProcessPool, ThreadPool
    from tqdm import tqdm
    # 冻结静态参数
    p_func = partial(func, **kwargs)
    # 打开进程/线程池
    pool = ThreadPool() if thread else ProcessPool()
    try:
        if show:
            start = time.time()
            # imap方法
            with tqdm(total=len(args[0]), desc="计算进度") as t:  # 进度条设置
                r = []
                for i in pool.imap(p_func, *args):
                    r.append(i)
                    t.set_postfix({'并行函数': func.__name__, "计算花销": "%ds" % (time.time() - start)})
                    t.update()
        else:
            # map方法
            r = pool.map(p_func, *args)
        return r
    except Exception as e:
        print(e)
    finally:
        # 关闭池
        pool.close()  # close the pool to any new jobs
        pool.join()  # cleanup the closed worker processes
        pool.clear()  # Remove server with matching state

In [33]:
# run model
## define model parameters
dependcon = [
    {'v': 'condition','a': 'condition','t': 'condition'},
    {'v': 'condition','a': 'condition'},
    {'v': 'condition','t': 'condition'},
    {'v': 'condition'}]

parallel(run_model,dependcon,df=df)

/opt/conda/lib/python3.7/site-packages/scipy/optimize/optimize.py:1960: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/optimize.py:1960: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/optimize.py:1960: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)
/opt/conda/lib/python3.7/site-packages/scipy/optimize/optimize.py:1960: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


 [-----------------95%----------------  ] 475 of 500 complete in 501.7 secvusage time: 9.085 min
 [-----------------83%-----------       ] 416 of 500 complete in 501.5 secvausage time: 9.447 min
 [-----------------96%----------------  ] 481 of 500 complete in 540.6 secvtusage time: 10.101 min
 [-----------------100%-----------------] 501 of 500 complete in 549.4 secvatusage time: 10.263 min


In [35]:
import hddm
[i.get_group_nodes() for i in m]

AttributeError: 'NoneType' object has no attribute 'get_group_nodes'

AttributeError: 'NoneType' object has no attribute 'get_group_nodes'